In [1]:
# 17-09-2024 The code below is a combination of already existing code by R. Buring, and adjustments made by F. Schipper 
# (particularly, the biggest adjustments are checking that the basis element(s) of the cocycle space are Hamiltonian, 
# commentary and printing of output). It is supplementary material accompanying proceedings written for the ISQS28 (Integrable 
#Systems and Quantum Symmetries) conference (see also: arXiv:2409.15932).

# We import the following (see https://github.com/rburing/gcaops) to be able to run the code.
from gcaops.graph.formality_graph import FormalityGraph
from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra
from gcaops.graph.undirected_graph_complex import UndirectedGraphComplex
from gcaops.graph.directed_graph_complex import DirectedGraphComplex

# These are the encodings of the 14 Kontsevich graphs on 3 vertices and 1 sink in 2D.
two_d_graphs="(0,1;2,3;1,3)+(0,1;1,2;1,3)+(0,3;2,3;2,3)+(0,3;2,3;1,3)+(0,2;2,3;1,3)+(0,3;1,2;1,3)+(0,3;2,3;1,2)+(0,3;1,2;1,2)+(0,2;2,3;1,2)+(0,2;1,2;1,2)+(0,1;1,3;1,2)+(0,3;1,3;1,2)+(0,1;1,3;2,3)+(0,1;1,3;1,3)"

# To move from the encodings of the graphs to actual graphs, we use the next function. The function splits the 
# encoding by vertex via ;, and then the target vertices by ,. A graph is returned on 3 vertices, 1 sink, and with edges
# (origin vertex, target vertex). As an example, the first encoding (0,1;2,3;1,3) correspond to a graph with 1 sink (vertex 0),
# and 3 regular vertices (vertices 1,2,3), with 6 edges (1,0), (1,1), (2,2), (2,3), (3,1), (3,3).
def encoding_to_graph(encoding):
    targets = [tuple(int(v) for v in t.split(',')) for t in encoding[1:-1].split(";")] 
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(1, 3, edges) 
    
# Split the encodings and compute the corresponding graphs
encodings = two_d_graphs.split("+") 
graphs = [encoding_to_graph(e) for e in encodings]
print('We have', len(graphs), 'graphs.\n')

# Create the differential polynomial ring. We are working in 2D, so we only have even coordinates x,y and the corresponding 
# odd coordinates xi[0] and xi[1]. rho is exactly the rho in a 2D Poisson bracket (P= rho dx dy). Finally, 
# max_differential_orders tells the programme how many times rho can be differentiated. The maximum is stipulated by the graphs, 
# as we cannot have double edges. Thus, the maximum in degree of each vertex is 3. As we will be looking at [[P,X]], we add
# an extra +1 to the differential orders since taking the Schouten bracket with P introduces an extra derivative.
D2=DifferentialPolynomialRing(QQ,('rho', ), ('x','y'), max_differential_orders=[3+1])
rho, =D2.fibre_variables()
x,y= D2.base_variables()
even_coords=[x,y]

S2.<xi0,xi1>=SuperfunctionAlgebra(D2, D2.base_variables())
xi=S2.gens()
odd_coords=xi

# We now compute the vector fields corresponding to the graphs. E is the Euler vector field in the sink (vertex 0), and
# epsilon is the Levi-Civita tensor. Note that we have a Levi-Civita tensor at each of the vertices 1, 2, 3. We first compute
# the sign of each term appearing in the formulas, and then compute the differential polynomial.
X_vector_fields=[]
E=x*xi[0]+y*xi[1] 
epsilon = xi[0]*xi[1] 
import itertools
for g in graphs:
    term = S2.zero()
    for index_choice in itertools.product(itertools.permutations(range(2)), repeat=3): 
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]* epsilon[index_choice[2]]
        vertex_content = [E, S2(rho), S2(rho), S2(rho)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].diff(even_coords[index])
        term += sign * prod(vertex_content)
    X_vector_fields.append(term)

# We check how many (if any) graphs evaluate to 0. 
zeros=X_vector_fields.count(0)
print('There are', zeros, 'graphs that evaluate to 0 under the morhpism from graphs to multivectors.\n')

# In case the are graphs that evaluate to 0, the line below shows which graphs do so (note that the counting starts from 1!).
#[k+1 for (k,X) in enumerate (X_vector_fields) if X==0]

# In 2D on only 3 vertices and 1 sink, the multivectors are pretty small and can be printed easily.
print('Here are the vector fields the graphs are evaluated into:')
for i in range(len(X_vector_fields)):
    print('graph', i+1,':', X_vector_fields[i])
print()

# The next part is to find out linear relations of the vector fields we just computed. We look at the monomials that appear
# in each xi[0] and xi[1] parts of the vector field, and store them in X_monomial_basis. 
X_monomial_basis = [set([]) for i in range(2)] 
for i in range(2): 
    for X in X_vector_fields:
        X_monomial_basis[i]|=set(X[i].monomials())
X_monomial_basis=[list(b) for b in X_monomial_basis]
X_monomial_index= [{m:k for k,m in enumerate(b)} for b in X_monomial_basis]
X_monomial_count= sum(len(b) for b in X_monomial_basis); X_monomial_count

# Next, we use this monomial basis to create a matrix that identifies each vector field by the monomials that appear in it.
X_evaluation_matrix= matrix(QQ, X_monomial_count, len(X_vector_fields), sparse=True)
for i in range(len(X_vector_fields)):
    v=vector(QQ, X_monomial_count, sparse=True)
    index_shift=0
    for j in range(2): 
        f=X_vector_fields[i][j]
        for coeff, monomial in zip(f.coefficients(), f.monomials()):
            monomial_index=X_monomial_index[j][monomial]
            v[index_shift+monomial_index]=coeff
        index_shift+=len(X_monomial_basis[j])
    X_evaluation_matrix.set_column(i,v)

# We can now detect linear relation by computing the nullity of this matrix
nullity = X_evaluation_matrix.right_nullity()
print('The vector fields have', nullity, 'linear relations among themselves.')
print('Claim 2: These relations are expressed by \n', X_evaluation_matrix.right_kernel(), '\n')

# We now compute the tetrahedral flow. First, we create the Poisson bivector P and check that it satisfies [[P,P]]=0
P= rho*epsilon 
if P.bracket(P)!=0:
    print('P is not a Poisson bivector. \n')

# Introduce the graph complex, and find the tetrahedron as a graph on 4 vertices and 6 edges in the cohomology. This 
# tetrahedron is unoriented, so we orient it. The next step is to make sure that the graph is built of wedges. This means 
# that there cannot be more than 2 outgoing edges at each vertex. This gives 2 graphs; one where 3 vertices have 2 outgoing
# edges, and one where 2 vertices have 2 outgoing edges and 2 vertices have 1 outgoing edge. 
# tetrahedron_oriented_filtered.show() gives a drawing of these graphs. 
# Finally, the bivector corresponding to the graph is computed.

GC=UndirectedGraphComplex(QQ, implementation='vector', sparse=True)
tetrahedron= GC.cohomology_basis(4,6)[0]
dGC=DirectedGraphComplex(QQ, implementation='vector')
tetrahedron_oriented= dGC(tetrahedron)
tetrahedron_oriented_filtered= tetrahedron_oriented.filter(max_out_degree=2)
# tetrahedron_oriented_filtered.show()
tetrahedron_operation= S2.graph_operation(tetrahedron_oriented_filtered)
Q_tetra= tetrahedron_operation(P, P, P, P) /8
print('The tetrahedral flow in 2D is', Q_tetra, '\n')

# Now that we have the tetrahedral flow, we see if we can create a vectorfield X from our previously computed 
# graphs-to-vector fields such that [[P,X]]= Q_tetra. We first look which bivectors X_bivectors the vector fields become 
# after taking the Schouten bracket with P.
X_bivectors=[]
for X in X_vector_fields:
    X_bivectors.append(P.bracket(X))
    
zero_bivectors = X_bivectors.count(0)
print('There are', zero_bivectors, 'vector fields in X_vector_fields that evaluate to 0 bivectors after taking the Schouten bracket with P.')
print('These vector fields that evaluate to 0 are obtained from the graphs:')
for (k,X) in enumerate(X_bivectors):
    if X==0:
        print('graph', k+1,)
print()

# Now, we extract the monomials appearing in these bivectors (as well as in Q_tetra).
Q_monomial_basis={}
from itertools import combinations
for i,j in combinations(range(2),2):
    Q_monomial_basis[i,j]=set(Q_tetra[i,j].monomials())
    for P_X in X_bivectors:
        Q_monomial_basis[i,j]|= set(P_X[i,j].monomials())
        
Q_monomial_basis={idx: list(b) for idx, b in Q_monomial_basis.items()}
Q_monomial_index= {idx:{m:k for k,m in enumerate(b)} for idx, b in Q_monomial_basis.items()}
Q_monomial_count=sum(len(b) for b in Q_monomial_basis.values());

# We create the vector representation of Q_tetra in terms of the monomials.
Q_tetra_vector= vector(QQ, Q_monomial_count, sparse=True)
index_shift=0
for i,j in Q_monomial_basis:
    for coeff, monomial in Q_tetra[i,j]:
        monomial_index= Q_monomial_index[i,j][monomial]
        Q_tetra_vector[monomial_index+index_shift]=coeff
    index_shift+=len(Q_monomial_basis[i,j])

# We create the matrix that represents the X_bivectors in terms of the monomials.
X_bivector_evaluation_matrix= matrix(QQ, Q_monomial_count, len(X_bivectors), sparse=True)
for k in range(len(X_bivectors)):
    P_X=X_bivectors[k]
    v=vector(QQ,Q_monomial_count, sparse=True)
    index_shift=0
    for i,j in Q_monomial_basis:
        for coeff, monomial in P_X[i,j]:
            monomial_index=Q_monomial_index[i,j][monomial]
            v[monomial_index+index_shift]=coeff
        index_shift+=len(Q_monomial_basis[i,j])
    X_bivector_evaluation_matrix.set_column(k,v)

# We solve the linear system.
X_solution=X_bivector_evaluation_matrix.solve_right(Q_tetra_vector)
print('Proposition 3: The solution on vector fields is given by', X_solution,' \n')
# So if we take 2*graph 1 +1*graph 2, evaluate this to a vector field X, then [[P, X]]= Q_tetra.
# Note that because of the linear relations, this solution is not unique on the level of graphs.

# Finally, we will check the homogeneous system. We look at the kernel of the X_bivector_evaluation_matrix and filter out the 
# nullity of the X_evaluation matrix
X_cocycle_space= X_bivector_evaluation_matrix.right_kernel().quotient(X_evaluation_matrix.right_kernel())
X_cocycles=[X_cocycle_space.lift(v) for v in X_cocycle_space.basis()]; X_cocycles
if all(X_bivector_evaluation_matrix*X_cocycle==0 for X_cocycle in X_cocycles)!= True:
    print('There is an error in the cocycle space.')
print('The space of solutions to the homogeneous system has dimension', X_cocycle_space.dimension(), )
print ('This shift is given by (Proposition 4) ', X_cocycles, 'and the corresponding vector field is', X_vector_fields[2], '\n')

# We check that this shift is induced by the Hamiltonian vector field
hamiltonian="(0,1;0,1)"

# We create a new encoding to graph definition, as this is a graph on 2 vertices and no sink.
def hamiltonian_encoding_to_graph(encoding):
    targets = [tuple(int(v) for v in t.split(',')) for t in encoding[1:-1].split(";")]
    edges = sum([[(k,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(0, 2, edges)
    
hamiltonian_graph= encoding_to_graph(hamiltonian)

# This computation is also slightly different; we do not have the Euler vector field since we do not have a sink, and as
# we have only 2 copies of the Poisson bivector, the index_choice for the sign is only on 2 repeats as well. Moreover, there
# are only 2 vertices to give vertex_content to. 
epsilon= xi[0]*xi[1]
import itertools
hamiltonian_formula=S2.zero()
for index_choice in itertools.product(itertools.permutations(range(2)),repeat=2):
    vertex_content = [S2(rho), S2(rho)]
    sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]
    for ((source, target), index) in zip(hamiltonian_graph.edges(), sum(map(list,index_choice), [])):
        vertex_content[target] = diff(vertex_content[target],even_coords[index])
    hamiltonian_formula += sign * prod(vertex_content)

hamiltonian_vector_field=P.bracket(hamiltonian_formula)
print('The Hamiltonian vector field is given by', hamiltonian_vector_field, '\n')

if hamiltonian_vector_field== 2*X_vector_fields[2]:
    print('Theorem 6: The shift in the cocycle space is given by 1/2 the Hamiltonian vector field.') 

We have 14 graphs.

There are 0 graphs that evaluate to 0 under the morhpism from graphs to multivectors.

Here are the vector fields the graphs are evaluated into:
graph 1 : (-rho_y*rho_xy^2 + rho_y*rho_xx*rho_yy)*xi0 + (rho_x*rho_xy^2 - rho_x*rho_xx*rho_yy)*xi1
graph 2 : (rho_y^2*rho_xxy - 2*rho_x*rho_y*rho_xyy + rho_x^2*rho_yyy)*xi0 + (-rho_y^2*rho_xxx + 2*rho_x*rho_y*rho_xxy - rho_x^2*rho_xyy)*xi1
graph 3 : (rho*rho_yy*rho_xxy - 2*rho*rho_xy*rho_xyy + rho*rho_xx*rho_yyy)*xi0 + (-rho*rho_yy*rho_xxx + 2*rho*rho_xy*rho_xxy - rho*rho_xx*rho_xyy)*xi1
graph 4 : (rho_y^2*rho_xxy - 2*rho_x*rho_y*rho_xyy + rho_x^2*rho_yyy)*xi0 + (-rho_y^2*rho_xxx + 2*rho_x*rho_y*rho_xxy - rho_x^2*rho_xyy)*xi1
graph 5 : (-rho_y*rho_xy^2 + rho_y*rho_xx*rho_yy)*xi0 + (rho_x*rho_xy^2 - rho_x*rho_xx*rho_yy)*xi1
graph 6 : (-rho_y*rho_xy^2 + rho_y*rho_xx*rho_yy)*xi0 + (rho_x*rho_xy^2 - rho_x*rho_xx*rho_yy)*xi1
graph 7 : (rho_y*rho_xy^2 - rho_y*rho_xx*rho_yy)*xi0 + (-rho_x*rho_xy^2 + rho_x*rho_xx*rho_yy)*xi1
graph 